# Importing libraries

In [34]:
import pdftotext
import pandas as pd
import string
import re
import os
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import gensim
import numpy as np
from sklearn import metrics

In [35]:
#!pip install gensim
#!pip install python-Levenshtein

# Load text from documents

In [36]:
CURRENT_PATH = os.getcwd()

# Load text from documents
def loadTextFromFile(directory, filenames, docs):
    trainingDirectory = CURRENT_PATH + "/" + directory
    for filename in os.listdir(trainingDirectory):
        filenames.append(int(filename[:-4]))  # Removes the .txt from the filename
        with open(trainingDirectory + "/" + filename, "r") as file:
            text = file.read()
        docs.append(text)


diversityInclusion = "diversityInclusion"
diversityInclusionFilenames = []
diversityInclusionDocs = []
diversityInclusionTestFilenames = []
diversityInclusionTestDocs = []
loadTextFromFile(
    diversityInclusion + "/" + "training",
    diversityInclusionFilenames,
    diversityInclusionDocs,
)
loadTextFromFile(
    diversityInclusion + "/" + "test",
    diversityInclusionTestFilenames,
    diversityInclusionTestDocs,
)


encourageGenders = "encourageGenders"
encourageGendersFilenames = []
encourageGendersDocs = []
encourageGendersTestFilenames = []
encourageGendersTestDocs = []
loadTextFromFile(
    encourageGenders + "/" + "training",
    encourageGendersFilenames,
    encourageGendersDocs,
)
loadTextFromFile(
    encourageGenders + "/" + "test",
    encourageGendersTestFilenames,
    encourageGendersTestDocs,
)

mentionOrgFeatures = "mentionOrgFeatures"
mentionOrgFeaturesFilenames = []
mentionOrgFeaturesDocs = []
mentionOrgFeaturesTestFilenames = []
mentionOrgFeaturesTestDocs = []
loadTextFromFile(
    mentionOrgFeatures + "/" + "training",
    mentionOrgFeaturesFilenames,
    mentionOrgFeaturesDocs,
)
loadTextFromFile(
    mentionOrgFeatures + "/" + "test",
    mentionOrgFeaturesTestFilenames,
    mentionOrgFeaturesTestDocs,
)

# Preparing a dataframe

In [37]:
# Builds a table with two columns: filename ('id'), and the text from the file (the job ad)

diversityInclusionData = pd.DataFrame()
diversityInclusionData["id"] = diversityInclusionFilenames
diversityInclusionData["text"] = diversityInclusionDocs
diversityInclusionTestData = pd.DataFrame()
diversityInclusionTestData["id"] = diversityInclusionTestFilenames
diversityInclusionTestData["text"] = diversityInclusionTestDocs

encourageGendersData = pd.DataFrame()
encourageGendersData["id"] = encourageGendersFilenames
encourageGendersData["text"] = encourageGendersDocs
encourageGendersTestData = pd.DataFrame()
encourageGendersTestData["id"] = encourageGendersTestFilenames
encourageGendersTestData["text"] = encourageGendersTestDocs

mentionOrgFeaturesData = pd.DataFrame()
mentionOrgFeaturesData["id"] = mentionOrgFeaturesFilenames
mentionOrgFeaturesData["text"] = mentionOrgFeaturesDocs
mentionOrgFeaturesTestData = pd.DataFrame()
mentionOrgFeaturesTestData["id"] = mentionOrgFeaturesTestFilenames
mentionOrgFeaturesTestData["text"] = mentionOrgFeaturesTestDocs

# Cleaning text

In [38]:
def clean_text(text):
    # Remove punctutation
    text = re.sub("[^a-zA-Z]", " ", text)
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    # Convert to lower
    text = text.lower()
    # Remove whitespaces
    text = " ".join(text.split())
    return text


diversityInclusionData["text"] = diversityInclusionData["text"].apply(
    lambda x: clean_text(x)
)
diversityInclusionTestData["text"] = diversityInclusionTestData["text"].apply(
    lambda x: clean_text(x)
)

encourageGendersTestData["text"] = encourageGendersData["text"].apply(
    lambda x: clean_text(x)
)
encourageGendersTestData["text"] = encourageGendersTestData["text"].apply(
    lambda x: clean_text(x)
)

mentionOrgFeaturesData["text"] = mentionOrgFeaturesData["text"].apply(
    lambda x: clean_text(x)
)
mentionOrgFeaturesTestData["text"] = mentionOrgFeaturesTestData["text"].apply(
    lambda x: clean_text(x)
)

# Remove stopwords

In [39]:
stop = set(stopwords.words("english"))

# A function to remove stopwords and short length words (< 2)
def remove_stopwords(text):
    new = []
    for word in text.split():
        if word not in stop and len(word) > 1:
            new.append(word)

    return " ".join(new)


diversityInclusionData["text"] = diversityInclusionData["text"].apply(
    lambda x: remove_stopwords(x)
)
diversityInclusionTestData["text"] = diversityInclusionTestData["text"].apply(
    lambda x: remove_stopwords(x)
)

encourageGendersData["text"] = encourageGendersData["text"].apply(
    lambda x: remove_stopwords(x)
)
encourageGendersTestData["text"] = encourageGendersTestData["text"].apply(
    lambda x: remove_stopwords(x)
)

mentionOrgFeaturesData["text"] = mentionOrgFeaturesData["text"].apply(
    lambda x: remove_stopwords(x)
)
mentionOrgFeaturesTestData["text"] = mentionOrgFeaturesTestData["text"].apply(
    lambda x: remove_stopwords(x)
)

# Reading pre-labeled target classes

In [40]:
diversityInclusionLabels = pd.read_csv(
    CURRENT_PATH + "/labels/diversityInclusion.csv"
)
diversityInclusionData = pd.merge(diversityInclusionData, diversityInclusionLabels)
diversityInclusionTestLabels = pd.read_csv(
    CURRENT_PATH + "/labels/diversityInclusionTest.csv"
)
diversityInclusionTestData = pd.merge(
    diversityInclusionTestData, diversityInclusionTestLabels
)


encourageGendersLabels = pd.read_csv(CURRENT_PATH + "/labels/encourageGenders.csv")
encourageGendersData = pd.merge(encourageGendersData, encourageGendersLabels)
encourageGendersTestLabels = pd.read_csv(
    CURRENT_PATH + "/labels/encourageGendersTest.csv"
)
encourageGendersTestData = pd.merge(
    encourageGendersTestData, encourageGendersTestLabels
)


mentionOrgFeaturesLabels = pd.read_csv(
    CURRENT_PATH + "/labels/mentionOrgFeatures.csv"
)
mentionOrgFeaturesData = pd.merge(mentionOrgFeaturesData, mentionOrgFeaturesLabels)
mentionOrgFeaturesTestLabels = pd.read_csv(
    CURRENT_PATH + "/labels/mentionOrgFeaturesTest.csv"
)
mentionOrgFeaturesTestData = pd.merge(
    mentionOrgFeaturesTestData, mentionOrgFeaturesTestLabels
)

# Using word2vec for manually training embeddings

In [41]:
# model = gensim.models.Word2Vec(
#         window = 10,
#         min_count = 2,
#         workers = 4
# )

# model.build_vocab(train['tokens'], progress_per=1000)

# model.epochs

# model.corpus_count

# model.train(train['text'], total_examples=model.corpus_count, epochs=model.epochs)

# model.save("./word-2-vec.model")

# model.wv.most_similar("male")

# Using pre-trained Glove word embeddings

In [42]:
from gensim.scripts.glove2word2vec import glove2word2vec

# need to download the model from https://nlp.stanford.edu/projects/glove/
# then add to directory
glove_path = 'glove.twitter.27B.100d.txt'
word2vec_output_file = 'glove-100d'+'.word2vec'

glove2word2vec(glove_path, word2vec_output_file)

/var/folders/91/nvbg7bb11cq2_8ybkhv_9chw0000gn/T/ipykernel_2087/3076838374.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_path, word2vec_output_file)


(1193514, 100)

In [43]:
from gensim.models import KeyedVectors

# load the GloVe model
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [44]:
#Show a word embedding
print('King: ',model.get_vector('king'))

result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

print('Most similar word to King + Woman: ', result)

King:  [-3.7500e-01 -2.7532e-01  1.2489e-01 -9.2143e-02 -4.3104e-01  2.5268e-02
 -4.1867e-02  1.2848e-01 -7.9363e-02 -1.0011e-01  1.4076e-01  1.0922e-01
 -3.4546e+00 -6.9851e-01  6.6580e-01  5.1494e-01  4.5912e-01 -2.1957e-01
  4.4094e-01 -3.0631e-01  1.2293e-01 -9.9830e-02 -2.5755e-01 -6.1872e-01
  1.0613e+00 -9.4278e-01  1.9284e-01 -8.2089e-02  2.7782e-01 -1.8595e-01
  2.9140e-02 -3.0870e-01 -3.9870e-01 -4.3038e-01  3.8403e-01  3.3243e-01
 -1.4446e-01  1.6682e-01  4.2301e-01 -2.6490e-01 -7.8106e-02 -4.6756e-01
 -3.4039e-01 -1.3690e-01  7.0890e-01 -4.8015e-01  8.9183e-02 -2.3709e-01
  7.5124e-01  2.0507e-01 -5.5263e-01 -3.8105e-01 -7.7082e-02  3.6118e-01
 -8.9840e-01 -5.3537e-01  3.3161e-01 -1.3460e-01 -5.7742e-02  1.9428e-01
  1.8008e-01 -4.0697e-01  2.6654e-03 -7.8771e-02 -2.3616e-01 -9.8115e-01
 -1.6823e-01  1.1459e-01 -2.7011e-01 -2.1435e-02  2.3491e-01 -1.1341e+00
 -3.3837e-01  1.6548e-01  5.3073e-01 -3.0098e-01 -3.6769e-01  4.2092e-01
  1.4201e-01  1.7346e-02  7.8406e-01  3.1441

In [45]:
# Find similar words to diversity
model.most_similar('diversity')

[('inclusion', 0.7886667847633362),
 ('sustainability', 0.744009256362915),
 ('equality', 0.7191735506057739),
 ('empowerment', 0.717718780040741),
 ('unity', 0.6994752883911133),
 ('leadership', 0.6993807554244995),
 ('advocacy', 0.696628749370575),
 ('innovation', 0.6965843439102173),
 ('initiative', 0.686913013458252),
 ('environmental', 0.685417115688324)]

# Creating a class for vectorizing the sentences

Source: https://edumunozsala.github.io/BlogEms/jupyter/nlp/classification/embeddings/python/2020/08/15/Intro_NLP_WordEmbeddings_Classification.html

In [46]:
model.vector_size

100

In [47]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
#     v = self.word_vectors.get_vector('king')
#     self.D = v.shape[0]
    self.D = model.vector_size

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [48]:
# Set a word vectorizer
vectorizer = Word2VecVectorizer(model)

Loading in word vectors...
Finished loading in word vectors


# Embedding each document and splitting into train test data

In [49]:
# labels = ['target1', 'target2', 'target3']

# text_vectors = vectorizer.fit_transform(data['text'])    
# target_labels = data[labels]
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(text_vectors, target_labels, test_size=0.2, random_state = 200) 

# models = []

# for i in range(3):
    
#     rf = RandomForestClassifier(n_estimators=200)
#     models.append(rf.fit(Xtrain, Ytrain[labels[i]]))

# predictions = []
# for model in models:
#     predictions.append(model.predict(Xtest))

# for i in range(3):
#     print('\t Classification report for', labels[i], '\n')
#     print(metrics.classification_report(Ytest[labels[i]], predictions[i],  digits=5))
# #     plot_confusion_matrix(Ytest[i], predictions[i])
# #     plot_roc_curve(Ytest[i], predictions[i])

# Ytrain['target1'].value_counts() .......

# Splitting data & implementing bag of words

        ### MY TEST ###
diversityInclusionText = vectorizer.fit_transform(
    diversityInclusionData["text"]
)
diversityInclusionLabels = np.array(diversityInclusionData[diversityInclusion])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(diversityInclusionText, diversityInclusionLabels, test_size=0.2, random_state = 200) 
# diversityInclusionTextFeatures = diversityInclusionText.toarray()
print(Ytrain[0])

models = []

for i in range(3):
    
    rf = RandomForestClassifier(n_estimators=200)
    # print(rf.fit(Xtrain, Ytrain[i]))
    models.append(rf.fit(Xtrain, Ytrain[i]))

predictions = []
for model in models:
    predictions.append(model.predict(Xtest))

for i in range(3):
    print('\t Classification report for', labels[i], '\n')
    print(metrics.classification_report(Ytest[labels[i]], predictions[i],  digits=5))
#     plot_confusion_matrix(Ytest[i], predictions[i])
#     plot_roc_curve(Ytest[i], predictions[i])









diversityInclusionTestText = vectorizer.fit_transform(
    diversityInclusionTestData["text"]
)
diversityInclusionTestLabels = np.array(diversityInclusionTestData[diversityInclusion])
diversityInclusionTestTextFeatures = diversityInclusionTestText.toarray()

""""""

encourageGendersText = vectorizer.fit_transform(encourageGendersData["text"])
encourageGendersLabels = np.array(encourageGendersData[encourageGenders])
encourageGendersTextFeatures = encourageGendersText.toarray()

encourageGendersTestText = vectorizer.fit_transform(
    encourageGendersTestData["text"]
)
encourageGendersTestLabels = np.array(encourageGendersTestData[encourageGenders])
encourageGendersTestTextFeatures = encourageGendersTestText.toarray()

""""""

mentionOrgFeaturesText = vectorizer.fit_transform(
    mentionOrgFeaturesData["text"]
)
mentionOrgFeaturesLabels = np.array(mentionOrgFeaturesData[mentionOrgFeatures])
mentionOrgFeaturesTextFeatures = mentionOrgFeaturesText.toarray()

mentionOrgFeaturesTestText = vectorizer.fit_transform(
    mentionOrgFeaturesTestData["text"]
)
mentionOrgFeaturesTestLabels = np.array(mentionOrgFeaturesTestData[mentionOrgFeatures])
mentionOrgFeaturesTestTextFeatures = mentionOrgFeaturesTestText.toarray()



trainingData = [
    [diversityInclusionTextFeatures, diversityInclusionLabels],
    [encourageGendersTextFeatures, encourageGendersLabels],
    [mentionOrgFeaturesTextFeatures, mentionOrgFeaturesLabels],
]

Numer of samples with no words found: 0 / 2
1


TypeError: Singleton array array(1) cannot be considered a valid collection.

# Creating and evluating a machine learning model 

In [ ]:
models = []

for i in range(3):
    
    rf = RandomForestClassifier(n_estimators=200)
    models.append(rf.fit(trainingData[i][0], trainingData[i][1]))

In [ ]:
Ytrain['target1'].value_counts()

0    97
1    63
Name: target1, dtype: int64

In [ ]:
Ytrain['target2'].value_counts()

0    114
1     46
Name: target2, dtype: int64

In [ ]:
Ytrain['target3'].value_counts()

0    82
1    78
Name: target3, dtype: int64

In [ ]:
Ytest['target1'].value_counts()

0    22
1    18
Name: target1, dtype: int64

In [ ]:
Ytest['target2'].value_counts()

0    25
1    15
Name: target2, dtype: int64

In [ ]:
Ytest['target3'].value_counts()

1    23
0    17
Name: target3, dtype: int64